In [3]:
# Import dependencies
import pandas as pd
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

ModuleNotFoundError: No module named 'geopy'

In [ ]:
# Read CSV data
CVD_data = pd.read_csv('Averaged_Cleaned_filtered_data.csv')
CVD_data.head(10)

In [ ]:
# Cache to store lat/lon for state abbreviations
cache = {}

# Function to get latitude and longitude for a given state abbreviation with retries
def get_lat_lon(state_abbr):
    if state_abbr in cache:
        return cache[state_abbr]

    geolocator = Nominatim(user_agent="my_geocoder")
    attempts = 0
    while attempts < 5:
        try:
            location = geolocator.geocode(state_abbr + ", USA")
            if location:
                cache[state_abbr] = (location.latitude, location.longitude)
                return location.latitude, location.longitude
            else:
                cache[state_abbr] = (None, None)
                return None, None
        except (GeocoderTimedOut, GeocoderServiceError):
            attempts += 1
            time.sleep(2 ** attempts)  # Exponential backoff
    cache[state_abbr] = (None, None)
    return None, None

# Add latitude and longitude columns to the DataFrame
latitudes = []
longitudes = []

for state in CVD_data['State']:
    lat, lon = get_lat_lon(state)
    latitudes.append(lat)
    longitudes.append(lon)

CVD_data['Latitude'] = latitudes
CVD_data['Longitude'] = longitudes

In [ ]:
# Save the updated DataFrame to a new CSV file
CVD_data.to_csv('Averaged_Cleaned_filtered_data_with_lat_lon.csv', index=False)